In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import skimage.io
import skimage.filters
from skimage.filters import try_all_threshold
import matplotlib.image as mpimg
from scipy import ndimage as ndi
from skimage.transform import resize
import skimage.morphology
import skimage.feature
import math
from math import sqrt
from skimage.color import rgb2gray
from skimage import measure
from skimage import data, util

In [2]:
prefix = ['NT_48h_PI_']
regions = ['corpus_callosum', 'cortex', 'striatum']
images = 5

image_list = []
for pref in prefix:
    for region in regions:
        for num in range(1,images+1):
            image_list.append('{}{}_20x_images_XY{}.tif'.format(pref,region,num))

In [3]:
image_list

['NT_48h_PI_corpus_callosum_20x_images_XY1.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY2.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY3.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY4.tif',
 'NT_48h_PI_corpus_callosum_20x_images_XY5.tif',
 'NT_48h_PI_cortex_20x_images_XY1.tif',
 'NT_48h_PI_cortex_20x_images_XY2.tif',
 'NT_48h_PI_cortex_20x_images_XY3.tif',
 'NT_48h_PI_cortex_20x_images_XY4.tif',
 'NT_48h_PI_cortex_20x_images_XY5.tif',
 'NT_48h_PI_striatum_20x_images_XY1.tif',
 'NT_48h_PI_striatum_20x_images_XY2.tif',
 'NT_48h_PI_striatum_20x_images_XY3.tif',
 'NT_48h_PI_striatum_20x_images_XY4.tif',
 'NT_48h_PI_striatum_20x_images_XY5.tif']

In [4]:
output_data = pd.DataFrame(columns=['Image', 'DAPI count','DAPI intensity', 'DAPI area', 'PI count', 'PI intensity', 'PI area', 'PI intensity/DAPI count', 'PI w/in DAPI area', 'PI w/in DAPI/DAPI count', '%overlap PI/DAPI'])

In [5]:
output_data

,Image,DAPI count,DAPI intensity,DAPI area,PI count,PI intensity,PI area,PI intensity/DAPI count,PI w/in DAPI area,PI w/in DAPI/DAPI count,%overlap PI/DAPI


In [10]:
for image in image_list:
    img = skimage.io.imread('./NT_48h_PI/'+image)
    DAPI = img[0]
    PI = img[1]
    DAPI_filtered = ndi.gaussian_filter(DAPI, sigma=1)
    PI_filtered = ndi.gaussian_filter(PI, sigma=1)
    DAPI_thresh = skimage.filters.threshold_otsu(DAPI_filtered)
    DAPI_binary = DAPI_filtered > DAPI_thresh
    PI_thresh = skimage.filters.threshold_otsu(PI_filtered)
    PI_binary = PI_filtered > PI_thresh
    DAPI_dilation = skimage.morphology.binary_dilation(DAPI_binary)
    DAPI_filled_holes = ndi.binary_fill_holes(DAPI_dilation)
    DAPI_clean = skimage.morphology.remove_small_objects(DAPI_filled_holes, min_size=30)
    PI_dilation = skimage.morphology.binary_dilation(PI_binary)
    PI_filled_holes = ndi.binary_fill_holes(PI_dilation)
    PI_clean = skimage.morphology.remove_small_objects(PI_filled_holes, min_size=30)
    labeled_DAPI, _ = ndi.label(DAPI_clean)
    labeled_PI, _ = ndi.label(PI_clean)
    PI_props = measure.regionprops(labeled_PI)
    DAPI_props = measure.regionprops(labeled_DAPI)
    
    props_DAPI = pd.DataFrame(columns=['centroid x', 'centroid y', 'filled_area', 'area'])

    k = 1
    for props in DAPI_props:
    #Get the properties that I need for areas
    #Add them into a pandas dataframe that has the same number of rows as objects detected
        centroid = props.centroid
        centroid_x = centroid[0]
        centroid_y = centroid[1]
        fill_area = props.filled_area
        area = props.area
        props_DAPI.loc[k] = [centroid_x, centroid_y, fill_area, area]
        k = k + 1
        

    props_PI = pd.DataFrame(columns=['centroid x', 'centroid y', 'filled_area', 'area'])

    j = 1
    for props in PI_props:
    #Get the properties that I need for areas
    #Add them into a pandas dataframe that has the same number of rows as objects detected
        centroid = props.centroid
        centroid_x = centroid[0]
        centroid_y = centroid[1]
        fill_area = props.filled_area
        area = props.area
        props_PI.loc[j] = [centroid_x, centroid_y, fill_area, area]
        j = j + 1
    
    PI_overlap_sum = 0
    DAPI_pixels = 0
    i = 0
    PI_pixels = 0

    for ydim in range(0,512):
        for xdim in range(0,512):
            if labeled_DAPI[ydim][xdim] != 0:
                PI_overlap_sum = PI_overlap_sum + PI[ydim][xdim]
                DAPI_pixels = DAPI_pixels+1
                i = i + 1
                if labeled_PI[ydim][xdim] != 0:
                    PI_pixels = PI_pixels + 1
    
    output_data = output_data.append({'Image':image, 'DAPI count':len(props_DAPI), 'PI count':len(props_PI), 'DAPI intensity':DAPI.sum(), 'PI intensity':PI.sum(), 'DAPI area':props_DAPI['area'].sum(), 'PI area':props_PI['area'].sum(), 'PI intensity/DAPI count':PI.sum()/len(props_DAPI), 'PI w/in DAPI area':PI_overlap_sum, 'PI w/in DAPI/DAPI count':PI_overlap_sum/len(props_DAPI), '%overlap PI/DAPI':PI_pixels/DAPI_pixels}, ignore_index=True)

In [11]:
output_data

,Image,DAPI count,DAPI intensity,DAPI area,PI count,PI intensity,PI area,PI intensity/DAPI count,PI w/in DAPI area,PI w/in DAPI/DAPI count,%overlap PI/DAPI
0,NT_48h_PI_corpus_callosum_20x_images_XY1.tif,315,70720434,18046.0,290,7147676,16695.0,22691.034921,1735889,5510.758730,0.836806
1,NT_48h_PI_corpus_callosum_20x_images_XY2.tif,273,84315507,16357.0,271,8048631,16727.0,29482.164835,1805426,6613.282051,0.871859
2,NT_48h_PI_corpus_callosum_20x_images_XY3.tif,225,88488065,16981.0,228,8389051,17574.0,37284.671111,1765586,7847.048889,0.891644
3,NT_48h_PI_corpus_callosum_20x_images_XY4.tif,301,90817240,20498.0,297,9429939,21988.0,31328.700997,2189869,7275.312292,0.894038
4,NT_48h_PI_corpus_callosum_20x_images_XY5.tif,259,96736673,16530.0,252,8433973,18334.0,32563.602317,1526964,5895.613900,0.881065
5,NT_48h_PI_cortex_20x_images_XY1.tif,320,79038830,18223.0,299,9343973,17065.0,29199.915625,1571111,4909.721875,0.821654
6,NT_48h_PI_cortex_20x_images_XY2.tif,191,48793383,9754.0,200,5653584,9789.0,29599.916230,771313,4038.287958,0.774862
7,NT_48h_PI_cortex_20x_images_XY3.tif,299,77405892,16324.0,268,9056161,14557.0,30288.163880,1653463,5529.976589,0.816160
8,NT_48h_PI_cortex_20x_images_XY4.tif,296,79536462,16642.0,278,10731255,15597.0,36254.239865,1841723,6222.037162,0.853263
9,NT_48h_PI_cortex_20x_images_XY5.tif,296,79654460,16094.0,278,9785963,15164.0,33060.685811,1664994,5624.979730,0.849571


In [12]:
output_data.to_csv('1_25_20_NT_48h_PI_quantification.csv', encoding='utf-8', index=False)